In [19]:
#Die notwendigen libraries importieren.

import numpy as np
import pandas as pd
from datetime import datetime as dt

In [20]:
# Die Daten von einer .CSV Datei in einer DF hinzufügen.

loc = "C:/Users/thine/Desktop/Football-Data-Analysis-and-Prediction-master/Datasets/"

data_1 = pd.read_csv(loc + '2017-18.csv')
data_2 = pd.read_csv(loc + '2018-19.csv')
data_3 = pd.read_csv(loc + '2019-20.csv')
data_4 = pd.read_csv(loc + '2020-21.csv')
data_5 = pd.read_csv(loc + '2021-22.csv')

In [21]:
#Die Daten als Zeit gliedern. 
   
def parse_date_other(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%d/%m/%Y').date()
    

data_1.Date = data_1.Date.apply(parse_date_other)    
data_2.Date = data_2.Date.apply(parse_date_other)    
data_3.Date = data_3.Date.apply(parse_date_other)           
data_4.Date = data_4.Date.apply(parse_date_other)    
data_5.Date = data_5.Date.apply(parse_date_other)    


#Die entsprechenden Statistiken zu den Spielen holen.
                      
columns_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'] #FTR - Full Time Result

statistics_1 = data_1[columns_req]                      
statistics_2 = data_2[columns_req]
statistics_3 = data_3[columns_req]
statistics_4 = data_4[columns_req]
statistics_5 = data_5[columns_req]

In [22]:
# Die Goals nach Teams und Matchwoche aggregiert.

def get_goals_scored(statistic):
    
    # Hierbei wird eine Dictionary erstelle, wobei die Teams als Keys gespeichert werden.
    teams = {}
    for i in statistic.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # Die Value der Keys sind die Ortschaften der Spiele.
    for i in range(len(statistic)):
        HTGS = statistic.iloc[i]['FTHG'] #FTHG - Full Time Home Goals
        ATGS = statistic.iloc[i]['FTAG'] #FTAG - Full Time Away Goals
        teams[statistic.iloc[i].HomeTeam].append(HTGS)
        teams[statistic.iloc[i].AwayTeam].append(ATGS)
    
    # Eine dataframe erstellen, wobei die Zeilen "Teams" und die Spalten "Matchwoche" sind.
    scoredGoals = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    scoredGoals[0] = 0
    
    # Bis zu dem Zeitpunkt aggregieren.
    for i in range(2,39):
        scoredGoals[i] = scoredGoals[i] + scoredGoals[i-1]
    return scoredGoals

In [24]:
def get_gss(statistic):
    GS = get_goals_scored(statistic)
   
    z = 0
    HTGS = []
    ATGS = []

    for i in range(380):
        HT = statistic.iloc[i].HomeTeam #
        AT = statistic.iloc[i].AwayTeam
        HTGS.append(GS.loc[HT][z])
        ATGS.append(GS.loc[AT][z])
        
        if ((i + 1)% 10) == 0:
            z = z + 1
        
    statistic['HTGS'] = HTGS #Home Team Goals Scored
    statistic['ATGS'] = ATGS #Away Team Goals Scored

    
    return statistic


# Für jede Datenset anwenden
statistics_1 = get_gss(statistics_1)
statistics_2 = get_gss(statistics_2)
statistics_3 = get_gss(statistics_3)
statistics_4 = get_gss(statistics_4)
statistics_5 = get_gss(statistics_5)

<ipython-input-24-3b901e655d2c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statistic['HTGS'] = HTGS #Home Team Goals Scored
<ipython-input-24-3b901e655d2c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statistic['ATGS'] = ATGS #Away Team Goals Scored


In [26]:
# Die entsprechenden Punktzahlen holen.

def get_points(result):
    if result == 'W': #Win = Gewonnen
        return 3
    elif result == 'D': #Draw = Unentschieden
        return 1
    else:
        return 0

In [27]:
def get_cuml_points(matchresult):
    matchresult_points = matchresult.applymap(get_points)
    for i in range(2,39):
        matchresult_points[i] = matchresult_points[i] + matchresult_points[i-1]
        
    matchresult_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchresult_points

In [28]:
def get_matchresult(statistic):
    
    # Eine Dictionary wird erstellt mit den Teams als Keys.
    teams = {}
    for i in statistic.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # Die Value der Keys ist eine Liste mit den Resultaten.
    for i in range(len(statistic)):
        if statistic.iloc[i].FTR == 'H':
            teams[statistic.iloc[i].HomeTeam].append('W')
            teams[statistic.iloc[i].AwayTeam].append('L')
        elif statistic.iloc[i].FTR == 'A':
            teams[statistic.iloc[i].AwayTeam].append('W')
            teams[statistic.iloc[i].HomeTeam].append('L')
        else:
            teams[statistic.iloc[i].AwayTeam].append('D')
            teams[statistic.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

In [30]:
def get_aggregate_points(statistic):
    matchresult = get_matchresult(statistic)
    cum_pts = get_cuml_points(matchresult)
    HTP = []
    ATP = []
    z = 0
    for i in range(380):
        HT = statistic.iloc[i].HomeTeam
        AT = statistic.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[HT][z])
        ATP.append(cum_pts.loc[AT][z])

        if ((i + 1)% 10) == 0:
            z = z + 1
            
    statistic['HTP'] = HTP
    statistic['ATP'] = ATP
    return statistic

# Für jede Datenset anwenden
statistics_1 = get_aggregate_points(statistics_1)
statistics_2 = get_aggregate_points(statistics_2)
statistics_3 = get_aggregate_points(statistics_3)
statistics_4 = get_aggregate_points(statistics_4)
statistics_5 = get_aggregate_points(statistics_5)

<ipython-input-30-fc90cca55666>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statistic['HTP'] = HTP
<ipython-input-30-fc90cca55666>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statistic['ATP'] = ATP


In [31]:
#Die Team form holen.

def get_form(statistic,num):
    form = get_matchresult(statistic)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        z = 0
        while z < num:
            form_final[i] += form[i-z]
            z += 1           
    return form_final

In [33]:
def add_form(statistic,num):
    form = get_form(statistic,num)
    H = ['M' for i in range(num * 10)]  
    A = ['M' for i in range(num * 10)]
    
    z = num
    for i in range((num*10),380):
        HT = statistic.iloc[i].HomeTeam
        AT = statistic.iloc[i].AwayTeam
        
        past = form.loc[HT][z]               
        H.append(past[num-1])                    
        
        past = form.loc[AT][z]               
        A.append(past[num-1])                   
        
        if ((i + 1)% 10) == 0:
            z = z + 1

    statistic['HM' + str(num)] = H                 
    statistic['AM' + str(num)] = A

    
    return statistic

In [34]:
def add_form_df(statistics):
    statistics = add_form(statistics,1)
    statistics = add_form(statistics,2)
    statistics = add_form(statistics,3)
    statistics = add_form(statistics,4)
    statistics = add_form(statistics,5)
    return statistics    
    
# Die Änderungen an die Datenframe anwenden
statistics_1 = add_form_df(statistics_1)
statistics_2 = add_form_df(statistics_2)
statistics_3 = add_form_df(statistics_3)
statistics_4 = add_form_df(statistics_4)
statistics_5 = add_form_df(statistics_5)

<ipython-input-33-4ff1b63182e4>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statistic['HM' + str(num)] = H
<ipython-input-33-4ff1b63182e4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statistic['AM' + str(num)] = A


In [35]:
#Die Spalten sortieren.

cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

statistics_1 = statistics_1[cols]
statistics_2 = statistics_2[cols]
statistics_3 = statistics_3[cols]
statistics_4 = statistics_4[cols]
statistics_5 = statistics_5[cols]

In [36]:
#Finale Datenframe

statistic = pd.concat([statistics_1,
                       statistics_2,
                       statistics_3,
                       statistics_4,
                       statistics_5],)


# Die Form Punktzahlen holen.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

statistic['HTFormPtsStr'] = statistic['HM1'] + statistic['HM2'] + statistic['HM3'] + statistic['HM4'] + statistic['HM5']
statistic['ATFormPtsStr'] = statistic['AM1'] + statistic['AM2'] + statistic['AM3'] + statistic['AM4'] + statistic['AM5']

statistic['HTFormPts'] = statistic['HTFormPtsStr'].apply(get_form_points)
statistic['ATFormPts'] = statistic['ATFormPtsStr'].apply(get_form_points)

statistic.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3',
       'AM4', 'AM5', 'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts', 'ATFormPts'],
      dtype='object')

In [37]:
def homewin(string):
    if string == 'H':
        return 1
    else:
        return 0
    
statistic['FTR'] = statistic.FTR.apply(homewin)

# Testing set (2019-20)
statistic_test = statistic[5700:]

#Die Finale Datenset sowie die Testset erstellen.
statistic.to_csv(loc + "final_dataset.csv")
statistic_test.to_csv(loc+"test.csv")